In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

import os

# Feature detectors

In [ ]:
from orb_sift_detectors import extract_keypoints_and_descriptors

In [ ]:
test_imgs_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                              'test_imgs')
image_path_1 = os.path.join(test_imgs_path,'cam2_1.jpg')
image_path_2 = os.path.join(test_imgs_path, 'cam1_1.jpg')

## ORB detector

In [ ]:
(kp1, d1), (kp2, d2) = extract_keypoints_and_descriptors(image_path_1,
                                                         image_path_2,
                                                         detector_type='ORB')

In [ ]:
img1 = cv.drawKeypoints(cv.imread(image_path_1, cv.IMREAD_GRAYSCALE), 
                        kp1, 
                        None, 
                        color=(0,255,0), 
                        flags=cv.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
img2 = cv.drawKeypoints(cv.imread(image_path_2, cv.IMREAD_GRAYSCALE), 
                        kp1, 
                        None, 
                        color=(0,255,0), 
                        flags=cv.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img2)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img1)
plt.axis('off')

plt.tight_layout()
plt.show()

## SIFT detector

In [ ]:
(kp1, d1), (kp2, d2) = extract_keypoints_and_descriptors(image_path_1,
                                                         image_path_2,
                                                         detector_type='SIFT')

In [ ]:
img1 = cv.drawKeypoints(cv.imread(image_path_1, cv.IMREAD_GRAYSCALE), 
                        kp1, 
                        None, 
                        color=(0,255,0), 
                        flags=cv.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
img2 = cv.drawKeypoints(cv.imread(image_path_2, cv.IMREAD_GRAYSCALE), 
                        kp1, 
                        None, 
                        color=(0,255,0), 
                        flags=cv.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img2)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img1)
plt.axis('off')

plt.tight_layout()
plt.show()

## SuperPoint

In [ ]:
from transformers import AutoImageProcessor, SuperPointForKeypointDetection
import torch
from PIL import Image

In [ ]:
image_1 = Image.open(image_path_1).convert('RGB')
image_2 = Image.open(image_path_2).convert('RGB')

In [ ]:
images = [image_1, image_2]

processor = AutoImageProcessor.from_pretrained("magic-leap-community/superpoint")
model = SuperPointForKeypointDetection.from_pretrained("magic-leap-community/superpoint")

In [ ]:
model

In [ ]:
inputs = processor(images, return_tensors="pt")
outputs = model(**inputs)  # The outputs contain the list of keypoint coordinates with their 
                           # respective score and description (a 256-long vector).

In [ ]:
show_imgs = [None, None]

for i in range(len(images)):
    image_mask = outputs.mask[i]
    image_indices = torch.nonzero(image_mask).squeeze()
    image_keypoints = outputs.keypoints[i][image_indices]
    image_scores = outputs.scores[i][image_indices]
    image_descriptors = outputs.descriptors[i][image_indices]
    
    image_np = np.transpose(inputs['pixel_values'][i], (1, 2, 0)).numpy()

    # Ensure the image is contiguous and in uint8 format (0-255 range)
    if image_np.max() <= 1.0:
        image_np = (image_np * 255).astype(np.uint8)
    else:
        image_np = image_np.astype(np.uint8)

    image_np = np.ascontiguousarray(image_np)
    
    for keypoint, score in zip(image_keypoints, image_scores):
        keypoint_x, keypoint_y = int(keypoint[0].item()), int(keypoint[1].item())
        color = (0, 0, 255)
        image_np = cv.circle(image_np, (keypoint_x, keypoint_y), 2, color, thickness=-1)
    
    show_imgs[i] = image_np

In [ ]:
image_1_rgb = cv.cvtColor(show_imgs[0], cv.COLOR_BGR2RGB)
image_2_rgb = cv.cvtColor(show_imgs[1], cv.COLOR_BGR2RGB)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image_1_rgb)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(image_2_rgb)
plt.axis('off')

plt.tight_layout()
plt.show()

## Summary

1. Использовать SIFT не будем, т.к. скорость его работы низка, да и как видно, при матчинге будет очень много несоответствий.  
2. ORB тоже показывает неудовлетворительные результаты, т.к. как видно он не отмечает все точки на объекте равномерно.  
3. Метод SuperPoint (на основе нейронной сети) работает значительно лучше - равномерность точек по объекту. Вдобавок к алгоритму поиска ключевых точек SuperPoint существует алгоритм матчинга (от тех же исследователей/разработчиков) который является логичным продолжением этого - [SuperGlue](https://github.com/magicleap/SuperGluePretrainedNetwork/tree/master)

Оставшиеся задачи:
1. Попробовать другие детекторы на основе DL (D2Net, R2D2)
2. Применить последовательно к SuperPoint алгоритм SuperGlue для матчинга точек
3. Также остается идея для использования сегментации чтобы отделить сам объект от фона и уже только на нем искать характеристические точки